In [1]:
#Geo
#from geopy.geocoders import Nominatim

#Postal Code
#!pip install pypostalcode
from pypostalcode import PostalCodeDatabase

#Tables
import pandas as pd

#Regex
import re

#Deepparse
# !pip install deepparse
from deepparse.parser import AddressParser #long import

# Clean Organization Data

In [2]:
dfOrg = pd.read_excel('../Data/Organizations.xlsx').iloc[:,1:]
dfFoundations = pd.read_excel('../Data/Foundations.xlsx').iloc[:,1:]

In [3]:
dfOrg.columns

Index(['ID', '2E CONTACT POUR', 'ADDRESS', 'AVIS', 'CONTACT', 'CONTRIBUTION',
       'COURRIEL', 'DDD', 'DOMAINE DINTERET', 'FAF', 'FAX', 'FILIALE DE',
       'ISFOUNDATION', 'LANGUE', 'LIMITES GEOG', 'N DE TEL', 'NAME',
       'NBRE DE SUCC', 'NOMBRE DEMPLOYES', 'NOTE', 'POSTE', 'PRINCIP FILIALES',
       'SECTEUR INDUSTRIEL', 'SITE WEB'],
      dtype='object')

In [4]:
def cleanUpSpaces(row_value):
    """
    Function to remove excess whitespaces
    """
    if type(row_value) == str:
        return ' '.join(row_value.split())

In [5]:
dfOrg.ADDRESS.isna().sum(), dfFoundations.ADDRESS.isna().sum()

(0, 0)

In [6]:
#Clean the spaces for ADDRESS
dfOrg.ADDRESS = dfOrg.ADDRESS.apply(lambda x: cleanUpSpaces(x))
dfFoundations.ADDRESS = dfFoundations.ADDRESS.apply(lambda x: cleanUpSpaces(x))

In [7]:
#Remove puncuation at start and end of strings
dfFoundations.NAME = dfFoundations.NAME.apply(lambda x: re.sub(r"(^[^\w]+)|([^\w]+$)", "", x))

### Address (DL Approach)

In [8]:
address_parser = AddressParser(model_type="bpemb", device=0) #Device = 0 means set to GPU, needs google colab

C:\Users\louis\anaconda3\envs\Scraping\lib\site-packages\deepparse\parser\address_parser.py:881: UserWarning: No CUDA device detected, device will be set to 'CPU'.
  warnings.warn("No CUDA device detected, device will be set to 'CPU'.")


Loading the embeddings model


In [9]:
org_parsed_addresses = address_parser(dfOrg.ADDRESS)
foundation_parsed_addresses = address_parser(dfFoundations.ADDRESS)

Vectorizing the address
Vectorizing the address


In [10]:
fields = ['StreetNumber', 'StreetName', 'Municipality', 'Province', 'PostalCode']
df_org_parsed_address = pd.DataFrame([org_parsed_address.to_dict(fields=fields) for org_parsed_address in org_parsed_addresses],
                                         columns=fields)
df_foundation_parsed_address = pd.DataFrame([foundation_parsed_address.to_dict(fields=fields) for foundation_parsed_address in foundation_parsed_addresses],
                                         columns=fields)

In [11]:
df_org_parsed_address['Province'] = df_org_parsed_address['Province'].str.upper()
df_org_parsed_address['PostalCode'] = df_org_parsed_address['PostalCode'].str.upper()
df_foundation_parsed_address['Province'] = df_foundation_parsed_address['Province'].str.upper()
df_foundation_parsed_address['PostalCode'] = df_foundation_parsed_address['PostalCode'].str.upper()

In [12]:
addressDicts = df_org_parsed_address.to_dict('records')

In [13]:
#Print real address and parsed in order to compare
for i, df_address in enumerate(dfOrg.ADDRESS):
    print(df_address)
    temp = addressDicts[i]
    print(f"{temp['StreetNumber']}, {temp['StreetName']}, {temp['Municipality']}, {temp['Province']}, {temp['PostalCode']}")
    print("\n")

7290, rue Frederick Banting Saint-Laurent QC H4T1Z2
7290, rue frederick, banting saint-laurent, QC, H4T1Z2


305, 12e Avenue Richelieu QC J3L3T2
305, 12e avenue, richelieu, QC, J3L3T2


3750, chemin de l'aéroport Trois-Rivières QC G9B2N8
3750, chemin de, l'aéroport trois-rivières, QC, G9B2N8


800 boul. Hymes Saint-Laurent QC H4S0B5
800, boul. hymes, saint-laurent, QC, H4S0B5


800 boul. Hymes Saint-Laurent QC H4S0B5
800, boul. hymes, saint-laurent, QC, H4S0B5


8401, rte Transcanadienne Saint-Laurent Québec H4S1Z12
8401, rte transcanadienne, saint-laurent, QUÉBEC, H4S1Z12


7950, Vauban Montréal QC H1J2X5
7950, vauban, montréal, QC, H1J2X5


75, rue Queen, bur. 6100 Montréal QC H3C2N6
75, rue queen, montréal, QC, H3C2N6


75, rue Queen, bur. 6100 Montréal QC H3C2N6
75, rue queen, montréal, QC, H3C2N6


2270, rue Garneau Longueuil QC J4G1E7
2270, rue garneau, longueuil, QC, J4G1E7


1331, rue Graham-Bell Boucherville QC J4B6A1
1331, rue graham-bell, boucherville, QC, J4B6A1


3000, rue

340, Route du Président-Kennedy St-Théophile QC G0M2A0
340, route du, président-kennedy st-théophile, QC, G0M2A0


2151, boul. Lapinière Brossard QC J4W2T5
2151, boul. lapinière, brossard, QC, J4W2T5


2721, Rte 341 Nord St-Jacques QC J0K2R0
2721, rte 341, nord st-jacques, QC, J0K2R0


267, boul. Sir-Wilfrid-Laurier Saint-Basile-le-Grand QC J3N0A4
267, boul. sir-wilfrid-laurier, saint-basile-le-grand, QC, J3N0A4


166, 4e avenue Montmagny QC G5V3L5
166, 4e avenue, montmagny, QC, G5V3L5


20, Côte-de-la-Fabrique Québec QC G1R3V9
20, côte-de-la-fabrique, québec, QC, G1R3V9


5905, rue Kieran Saint-Laurent QC H4S0A3
5905, rue kieran, saint-laurent, QC, H4S0A3


225, boul. Montcalm Nord Candiac QC J5R3L6
225, boul. montcalm, nord candiac, QC, J5R3L6


3075, chemin des Quatre-Bourgeois, bur. 200 Québec QC G1W5C4
3075, chemin des quatre-bourgeois, québec, QC, G1W5C4


2275, 107RueSaint-Georges-de-Beauce QC G5Y8G6
2275, 107ruesaint-georges-de-beauce g5y8g6, qc, None, None


900, boul. de Mais

In [14]:
#Merge street columns together
df_org_parsed_address['Street'] = df_org_parsed_address['StreetNumber'] +', ' + df_org_parsed_address['StreetName']
df_foundation_parsed_address['Street'] = df_foundation_parsed_address['StreetNumber'] +', ' + df_foundation_parsed_address['StreetName']

df_org_parsed_address = df_org_parsed_address.drop(['StreetNumber', 'StreetName'], axis=1)
df_foundation_parsed_address = df_foundation_parsed_address.drop(['StreetNumber', 'StreetName'], axis=1)

In [15]:
dfOrg = pd.concat([dfOrg, df_org_parsed_address], axis=1)
dfFoundations = pd.concat([dfFoundations, df_foundation_parsed_address], axis=1)

In [16]:
org_cols = dfOrg.columns.tolist()
foundation_cols = dfFoundations.columns.tolist()

moved_org_cols = org_cols[:4] + org_cols[-1:] + org_cols[-4:-1] + org_cols[4:-4]
moved_foundation_cols = foundation_cols[:4] + foundation_cols[-1:] + foundation_cols[-4:-1] + foundation_cols[4:-4]


In [17]:
dfOrg = dfOrg[org_cols]
dfFoundations = dfFoundations[foundation_cols]

In [18]:
dfOrg.sample(10)

,ID,2E CONTACT POUR,ADDRESS,AVIS,CONTACT,CONTRIBUTION,COURRIEL,DDD,DOMAINE DINTERET,FAF,...,NOMBRE DEMPLOYES,NOTE,POSTE,PRINCIP FILIALES,SECTEUR INDUSTRIEL,SITE WEB,Municipality,Province,PostalCode,Street
105,106,NaN,"10, rue Duke Montréal QC H3C2L7","Faire les demandes par : courrier, courriel, fax.","Madame Diane Desjardins, Adj. au Président",Dons & commandite,diane.desjardins@autodesk.com; med_ent@autodes...,En tout temps,"Arts, éducation, environnement et santé.",31 janvier,...,347,NaN,NaN,NaN,Informatique et logiciels; Logiciels d'animati...,www.autodesk.com,montréal,QC,H3C2L7,"10, rue duke"
32,33,NaN,"101, rue Fecteau Val-d'Or QC J9P0G4",Faire les demandes par : courrier.,"Madame Lyne Comtois, Coordonnatrice Exécutive",Dons & commandite,comtoisl@aircreebec.ca,En tout temps,Organismes communautaires.,31 décembre,...,152,NaN,2287,NaN,Transport; Transport aérien dans le domaine de...,www.aircreebec.ca,val-d'or,QC,J9P0G4,"101, rue fecteau"
692,693,NaN,1610 Industrial Avenue Port Coquitlam BC V3C6N3,Dons en « temps » (bénévolat). Faire les deman...,"Mr Mike Breed, President",Dons & commandite,infoeast@kamtechservices.com; infoeast@kamtech...,En tout temps,"Appuie le cancer du sein, organismes de bienfa...",NaN,...,NaN,NaN,NaN,NaN,Génie-conseil et produits d'ingénierie; Contra...,www.kamtechservices.com,port coquitlam,BC,V3C6N3,"1610, industrial avenue"
121,122,NaN,"Édifice de la BDC, 5, pl. Ville-Marie, bur. 10...","Faire les demandes par : courrier, courriel, fax.","Madame Annie Marsolais, Chef de la direction m...",Commandite,marketing-communications@bdc.ca; Social@BDC,En tout temps,Soutien aux entrepreneurs et contribue à crée...,31 mars,...,946,"Sur le site web sélectionner : À propos BDC, É...",NaN,NaN,Services financiers; Finances et services et p...,http://www.bdc.ca,pl. ville-marie bur. 100montréal,QC,H3B5E7,"5, édifice de la bdc"
390,391,NaN,"695, 90e Avenue LaSalle QC H8R3A4","Faire les demandes par : courrier, courriel, fax.","Monsieur Stéphane Lavigne, Vice-président & Ch...",Dons & commandite,stephane.lavigne@gdi.com,En tout temps,Environnement et général.,30 novembre,...,4 700,NaN,NaN,"Service d'entretien Distinction inc., Distinct...","Entretien, réparation, services; Service d'ent...",https://gdi.com/,lasalle,QC,H8R3A4,"695, 90e avenue"
1031,1032,NaN,"7900, boul. Henri-Bourassa O. Saint-Laurent QC...",Faire les demandes par : courriel.,"Monsieur Antoine Auclair, Vice-président Chef ...",Dons & commandite,investisseurs@richelieu.com; investisseurs@ric...,En tout temps,Causes charitables en général.,30 novembre,...,690,Envoyer votre demande de dons ou de commandite...,NaN,"Industries Cédan, Distributions 20-20, Richeli...",Quincaillerie spécialisée; Distribution de pro...,www.richelieu.com,o. saint-laurent,QC,H4S1V4,"7900, boul. henri-bourassa"
835,836,NaN,"9100, boul. Ray-Lawson Montréal QC H1J1K8",Dons en « temps » (bénévolat). Faire les deman...,"Madame Francine Malenfant, Représentante",Dons,fmalenfant@mip.ca,En tout temps,"Coopération internationale, éducation, hôpitau...",31 décembre,...,148,Sur le site web sélectionner : Responsabilité...,NaN,NaN,"Produits et services, santé et sécurité; Fabri...",http://www.mip.ca,montréal,QC,H1J1K8,"9100, boul. ray-lawson"
289,290,NaN,"1470, boul. Nobel Boucherville QC J4B5H3","Faire les demandes par : courrier, courriel.","Monsieur Pierre-Luc Jetté, Directeur général",Dons & commandite,jettep@cintas.com; info@cintas.com,En tout temps,"Événements, groupes et séries sportifs.",NaN,...,NaN,Aucun don ou commandite pour l'année 2019.,NaN,NaN,"Vêtements et chaussures, détail et gros; Fabri...",www.fr.cintas.ca,boucherville,QC,J4B5H3,"1470, boul. nobel"
956,957,NaN,"26, Wellington St. E., Bur. 700 Toronto ON M5E1S2",Dons aux org. suggérés par les employés. Faire...,Comité des dons,Dons,NaN,En tout temps,"Banque alimentaire, diabète juvénile, environn...",NaN,...,NaN,NaN,NaN,NaN,Services financiers; Société de courtage à ser...,https://www.holliswealth.com,700 toronto,ON,M5E1

In [19]:
dfFoundations.sample(10)

,ID,ACTIF,ADDRESS,AVIS,CATEGORIE,CONTACT,COURRIEL,DATE APPROB,DATE FIN DANN,DOMAINES DINTERET,...,OU,POSTE,PROJETS PRIVILEGIES,TEL,TOTAL ANNUEL,WEB,Municipality,Province,PostalCode,Street
52,847,13 800 $,"16 750, rte Transcanadienne Kirkland QC H9H4M7",Communiquer par : courrier ou fax. Favorise le...,Fonds d’employés,"Madame Joanna Ginocchi-Foster, Présidente",joanna.foster@merck.com,En tout temps,31 décembre,Causes charitables en général. Éducation. Enfa...,...,NaN,NaN,Fondations hospitalières. Recherches scientifi...,514-428-8675,33 500 $,NaN,transcanadienne kirkland,QC,H9H4M7,"16, 750 rte"
71,1191,NaN,"5960 Heisley Rd Mentor, OH, 44060-1834 USA",NaN,Fondation corporative,NaN,NaN,NaN,NaN,"Environnement, santé et sécurité industrielle.",...,NaN,NaN,Appui la santé et l'éducation scientifique et ...,NaN,NaN,www.steris.com,None,USA,44060-1834,"5960, heisley rd mentor oh"
11,201,1 500 000 $,"8500, pl. Marien Montréal QC H1B5W8",Aucun renseignement fourni (La fondation ne di...,Fondation privée,"Madame Marie-Berthe Des Groseillers, Présidente",braultetmartineau@fondation.com,NaN,31 décembre,Causes charitables en général (organismes enre...,...,NaN,2427.0,Campagne de financement. Dons jumelés (avec le...,514-648-4252,500 000 $,www.braultmartineau.com,montréal,QC,H1B5W8,"8500, pl. marien"
29,481,7 606 800 $,"189, boul. Hymus, bur. 601 Pointe-Claire, QC H...",NaN,Fondation corporative,NaN,NaN,En tout temps.,31 décembre,Bien être des enfants. Appui des enfants et de...,...,NaN,NaN,Soutient également différents organismes commu...,514 693.6460 1 855 693.6555,1 140 350 $,http://www.fondationbondepart.ca/,pointe-claire,QC,H9R1E9,"189, boul. hymus"
50,831,NaN,"8625, rte Transcanadienne Saint-Laurent QC H4S1Z6",Administre ses propres programmes (dans les ré...,Fondation corporative,"Madame Danièle Dufour, Directrice des communi...",communications@mckesson.ca,En tout temps,NaN,Causes charitables en général (organismes enre...,...,NaN,NaN,Bourses aux particuliers (50 bourses de 1 000 ...,514-745-2300,NaN,www.mckesson.ca/fr/communautaire/fondation. aspx,saint-laurent,QC,H4S1Z6,"8625, rte transcanadienne"
14,237,9 100 $,"de, Canadian Pacific Employee Charities Donati...","Communiquer par : courrier, courriel ou fax. L...",Fondation publique,"Madame Linda Falle, Secrétariat",donations@cpr.ca,En tout temps,31 décembre,Causes charitables en général. Centraide (du G...,...,linda_falle@cpr.ca,NaN,Dons jumelés (avec les employés et retraités).,514-395-5115,9 100 $,www.cpr.ca,bur. 401 montréal,QC,H3C3E4,"canadiens-de-montréal, de canadian pacific emp..."
73,1278,155 535 $,"1155, boul. René-Lévesque O., bur. 3200 Montré...",Communiquer par : courrier. Limites géographiq...,Fondation corporative,"Monsieur Christian Houle, Président",NaN,En tout temps,31 décembre,Causes charitables en général (organismes enre...,...,NaN,NaN,"Activité bénéfice annuelle (tournois, événemen...",NaN,1 173 800 :$,http://www.ultramarcst.ca -ou- https://www.ul...,montréal,QC,H3B0C9,"1155, boul. rené-lévesque o."
24,323,152 815 $,"La Fondation de la Corporation des 2335, rue G...",Administre ses propres programmes. Appuie à de...,Fondation corporative,"Madame Julie Lachance, Directrice",j.lachance@ccam.qc.ca,Trimestrielle,31 décembre,Aînés (services aux personnes âgées). Enfance ...,...,NaN,NaN,"Activité bénéfice annuelle (soirées-bénéfice, ...",514-331-2045,701 015 $,www.ccam.qc.ca,rue guénette saint-laurent,QC,H4R2E9,"2335, la fondation de la corporation des"
22,298,5 186 055 $,"La 1275, rue Saint-Antoine O. Montréal QC H3C5L2","Communiquer par : courrier, courriel ou fax. F...",Fondation publique,"Madame Bernadette Kajjouni, Directrice",fondation@canadiens.com,15 février et 15 septembre,30 juin,Causes charitables en général (organismes enre...,...,bkajjouni@centrebell.ca,NaN,Activité bénéfice annuelle (messages publicita...,514-989-2890,900 400 $,http://fondation.canadiens.com,saint-antoine o. montréal,QC,H3C5L2,"rue, la 1275"
7,121,953 750 $,"25 King St. W., 30 th Floor Co

### Industrial Sector Groupings

Methodology: Preprocess text (remove stopwords, etc), find the most common terms and filter those who are industrial sectors. Then group similar sectors into a unique term. Iterate back and forth to make sure majority of sectors fall into a category.

In [20]:
df_sents = dfOrg[['SECTEUR INDUSTRIEL']].rename(columns = {'SECTEUR INDUSTRIEL': 'SECTEUR_INDUSTRIEL'})

In [21]:
# nltk.download('wordnet')
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re

stopwords = stopwords.words('french') + stopwords.words('english') + ['/' ,'-', 'pour', 'ou', 'du', 'la', 'ou', 'de',
         'avec', 'par', 'depuis','a', 'd\'un', '']

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords] #french stopwords
    return " ".join(filtered_words)

In [22]:
df_sents.SECTEUR_INDUSTRIEL = df_sents.SECTEUR_INDUSTRIEL.apply(lambda s:preprocess(s))

In [23]:
sents_list = df_sents['SECTEUR_INDUSTRIEL'].tolist()

words = []
for sent in sents_list:
    words.extend(sent.split(' '))
df_words = pd.Series(words, name = 'words').to_frame()
sorted_words = df_words.words.value_counts().index.tolist()

In [24]:
df_sents['FLG'] = 0
cats = []
prev_nr_covered = 0
for w in sorted_words:
    df_sents['FLG'] += df_sents['SECTEUR_INDUSTRIEL'].apply(lambda x: w in x.split(' ')).astype(int)
    df_sents['FLG'] = df_sents['FLG'].apply(lambda x: min(1,x))
    nr_covered= df_sents['FLG'].sum()

    if nr_covered == prev_nr_covered:
        pass
    else:
        cats.append(w)

    if nr_covered == df_sents.shape[0]:
        break

    prev_nr_covered =   nr_covered

In [25]:
len(cats)

77

In [26]:
print(cats)

['services', 'produits', 'fabrication', 'financiers', 'fabricant', 'construction', 'transport', 'gestion', 'détail', 'alimentation', 'ingénierie', 'systèmes', 'vêtements', 'distribution', 'communications', 'commerce', 'bois', 'logiciels', 'etc', 'pièces', 'acier', 'entreprises', 'santé', 'conception', 'solutions', 'métal', 'distributeur', 'production', 'équipement', 'marketing', 'sécurité', 'assurances', 'vente', 'transformation', 'entretien', 'service', 'développement', 'exploitation', 'restauration', 'soins', 'offre', 'camions', 'plus', 'équipements', 'gamme', 'québec', 'centre', 'ameublement', 'bureau', 'canada', 'immobilier', 'secteurs', 'accessoires', 'manufacturier', 'fournisseur', 'assurance', 'affaires', 'réseau', 'environnement', 'technologies', 'administration', 'fabrique', 'commercialisation', 'agriculture', 'mondial', 'air', 'sport', 'ventilation', 'publiques', 'commerciale', 'divertissements', 'optique', 'système', 'imprimerie', 'enseignement', 'voyage', 'horticulture']


In [27]:
cats = ['fabrication', 'fabrique', 'fabricant',  'financiers',  'manufacturier',    'production', 'construction', 'transport', 'gestion', 'alimentation'
'ingénierie', 'systèmes', 'vêtements', 'distribution', 'communications',
 'commerce', 'bois', 'logiciels', 'acier',   'santé',  'métal', 'distributeur',
'équipement', 'marketing', 'sécurité', 'assurances',     'transformation', 'entretien', 'restauration', 'soins',  'camions',
   'ameublement', 'industriel', 'médias', 'immobilier',     'assurance',  'réseau', 'technologies', 'administration',  
    'commercialisation', 'commercial', 'agriculture', 'sport', 'voyage']
print(len(cats)) #after removing the terms that are not really industrial sectors, we have 43 categories to group.

43


In [28]:
# small script to verify how many companies would fall into a category of the selected list

df_sents['FLG'] = 0
prev_nr_covered = 0
for w in cats:
    df_sents['FLG'] += df_sents['SECTEUR_INDUSTRIEL'].apply(lambda x: w in x).astype(int)
    df_sents['FLG'] = df_sents['FLG'].apply(lambda x: min(1,x))
    nr_covered= df_sents['FLG'].sum()

    #print(nr_covered*100/1342)
print(nr_covered*100/1342)

90.38748137108793


In [29]:
# we write a dictionary to group into just 10 main sectors
dict_cats = { 
    'sector_fabrication' : ['fabrication', 'fabrique','fabricant', 'manufacturier', 'transformation','production','équipement'],
    'sector_financiers' : [ 'financiers'],
    'sector_commerce' : ['commercialisation', 'commercial', 'commerce','vêtements'],
    'sector_construction' : ['construction', 'acier','ingénierie','métal','bois', 'industriel','entretien', 'ameublement', 'immobilier'],
    'sector_transport':['transport', 'distribution', 'distributeur','camions','voyage'],
    'sector_gestion': [ 'gestion','administration', 'marketing'] ,
    'sector_systems': ['systèmes','logiciels', 'réseau',  'technologies', 'sécurité', 'communications', 'médias'],
    'sector_alimentation': [ 'alimentation','agriculture',     'restauration'  ],
    'sector_sante_assurance':[ 'santé', 'soins','assurances',     'assurance'  ]}

In [30]:
for key, value in dict_cats.items():
    df_sents[key] = df_sents['SECTEUR_INDUSTRIEL'].apply(lambda x: any(word in x.split(' ') for word in value) ).astype(int)

    

In [31]:
df_sents['sector_other'] = 1 - df_sents[list(dict_cats.keys())].max(axis=1)  #companies that don't fall into the created sectors, we give them a "others" category

In [32]:
df_sents[ list(dict_cats.keys())+['sector_other'] ].describe().loc['mean',]*100

sector_fabrication        39.493294
sector_financiers         10.059613
sector_commerce           11.028316
sector_construction       23.621461
sector_transport          18.628912
sector_gestion            11.922504
sector_systems            17.809240
sector_alimentation       11.326379
sector_sante_assurance     9.239940
sector_other               9.016393
Name: mean, dtype: float64

In [33]:
dfOrg = pd.concat( [dfOrg, df_sents.iloc[:,2:]], axis = 1)

### Interest Domains - Topic Extractions

Methodology: Preprocess text (remove stopwords, etc), feed the texts to an LDA and then assign highest scoring topic to each record.

In [34]:
df_corpus = dfOrg[['DOMAINE DINTERET']].rename(columns = {'DOMAINE DINTERET':'corpus1'})

In [35]:
df_corpus['corpus2'] = df_corpus.corpus1.apply(lambda s:preprocess(s)) 


In [36]:
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
fr = SnowballStemmer('french')

sentences = df_corpus['corpus2'].tolist()

new_sents = []
for s in sentences:
    new_sents.append(' '.join([fr.stem(word) for word in word_tokenize(s)]))

In [38]:
no_features = 100

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_features=no_features)
tf = tf_vectorizer.fit_transform(new_sents)
tf_feature_names = tf_vectorizer.get_feature_names_out()

no_topics = 6

# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=30, learning_method='online', random_state=0).fit(tf)

no_top_words = 20
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
général charit caus environ sant soutien humanitair organ carit pauvret autr enregistr projet appui éduc social sécur communaut contr sport
Topic 1:
enfant jeuness fondat centraid canad hôpital soutien montréal bours environ aid grand bienfais jeun famill etc canadien programm étud femm
Topic 2:
organ communautair local environ vi domain amélior pauvret qualit jeuness aîn activ carit aid collect sant servic seul social projet
Topic 3:
sant éduc environ sport cultur art sécur social développ centraid jeun communautair servic activ collect éven être bien recherch culturel
Topic 4:
privileg seul sant centraid environ organ emploi région roug croix travaillent vivent enfant communaut recyclag certain éduc hôpital habitat développ
Topic 5:
fondat canc malad societ sein québec canadien recherch coeur hôpital équip diabet enregistr vi centr oeuvr centraid carit contr programm


In [39]:
df_corpus = pd.concat( [ df_corpus, pd.DataFrame(lda.transform(tf)) ] , axis=1)

In [40]:
df_corpus.columns = ['corpus1', 'corpus2', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6']
df_corpus.iloc[:, 2:] = df_corpus.iloc[:, 2:].round(2)

In [41]:
pd.options.display.max_colwidth = 250

In [42]:
# we explore the high-scoring records in each topic to assess consistency

topic = 'topic_4'
df_corpus[['corpus2', topic]].drop_duplicates().sort_values(topic, ascending=False).head(30).sample(10)

,corpus2,topic_4
123,centraide développement jeunes éducation arts service communautaire santé prévention recherche,0.92
1197,arts culture coopération internationale éducation environnement santé bien être,0.91
583,culture développement régional éducation environnement développement durable santé sport partenaires socio économiques humanitaires,0.92
634,éducation perfectionnement main oeuvre protection environnement soins santé,0.86
629,centraide culture éducation environnement développement durable santé sciences secteur socio humanitaire sports,0.91
565,arts culture centraide éducation activités projets communautaires programme imagine programme héritage partager santé mieux être,0.93
1114,favoriser éducation environnement soutient projets construction écoles formation jeunes enseignants métiers création emplois secteur électricité pays développement,0.86
315,arts culture sport santé rayonnement développement affaires oeuvres humanitaires éducation,0.91
214,environnement sécurité insécurité routière culture populaire sécurité mobilité aînés sécurité jeunes conducteurs développement durable,0.91
1051,soutenir diverses activités événements caractère culturel éducatif social,0.86


In [43]:
# after review, we assign a topic name

df_corpus = df_corpus.rename(columns={'topic_1': 'general_charity'   ,
                          'topic_2': 'youth_health_education'  ,
                          'topic_3':  'community_environmental'  ,
                          'topic_4':   'health_arts_culture',
                          'topic_5':   'regional_specific',
                          'topic_6':   'cancer_disease_research',
})

In [44]:
df_corpus['main_topic'] = df_corpus[df_corpus.columns[2:8]].idxmax(axis=1)
df_corpus['main_topic_score'] = df_corpus[df_corpus.columns[2:8]].max(axis=1)

In [45]:
df_corpus['secondary_topic'] = df_corpus[df_corpus.columns[2:8]].apply(lambda x: x.nlargest(2).idxmin(), axis=1)

In [46]:
df_corpus['secondary_topic_scr'] = df_corpus[df_corpus.columns[2:8]].apply(lambda x: x.nlargest(2).min(), axis=1)

In [47]:
df_corpus.main_topic.value_counts(1)*100

general_charity            27.347243
health_arts_culture        25.186289
community_environmental    16.393443
youth_health_education     15.946349
regional_specific          10.283159
cancer_disease_research     4.843517
Name: main_topic, dtype: float64

In [48]:
dfOrg = pd.concat( [dfOrg,
                    df_corpus[['general_charity','health_arts_culture', 'community_environmental', 'youth_health_education',
                               'regional_specific', 'cancer_disease_research',
                               'corpus2','main_topic','main_topic_score', 'secondary_topic', 'secondary_topic_scr']] ],
                  axis=1).rename(columns = {'corpus2':'interest_domain_clean'})

In [49]:
dfOrg.columns = dfOrg.columns.str.upper()

In [50]:
####
# Save to xlsx
###
dfOrg.to_excel("../Data/Organizations_stg.xlsx", index=None)
dfFoundations.to_excel("../Data/Foundations_stg.xlsx", index=None)

In [51]:
dfOrg.columns 

Index(['ID', '2E CONTACT POUR', 'ADDRESS', 'AVIS', 'CONTACT', 'CONTRIBUTION',
       'COURRIEL', 'DDD', 'DOMAINE DINTERET', 'FAF', 'FAX', 'FILIALE DE',
       'ISFOUNDATION', 'LANGUE', 'LIMITES GEOG', 'N DE TEL', 'NAME',
       'NBRE DE SUCC', 'NOMBRE DEMPLOYES', 'NOTE', 'POSTE', 'PRINCIP FILIALES',
       'SECTEUR INDUSTRIEL', 'SITE WEB', 'MUNICIPALITY', 'PROVINCE',
       'POSTALCODE', 'STREET', 'SECTOR_FABRICATION', 'SECTOR_FINANCIERS',
       'SECTOR_COMMERCE', 'SECTOR_CONSTRUCTION', 'SECTOR_TRANSPORT',
       'SECTOR_GESTION', 'SECTOR_SYSTEMS', 'SECTOR_ALIMENTATION',
       'SECTOR_SANTE_ASSURANCE', 'SECTOR_OTHER', 'GENERAL_CHARITY',
       'HEALTH_ARTS_CULTURE', 'COMMUNITY_ENVIRONMENTAL',
       'YOUTH_HEALTH_EDUCATION', 'REGIONAL_SPECIFIC',
       'CANCER_DISEASE_RESEARCH', 'INTEREST_DOMAIN_CLEAN', 'MAIN_TOPIC',
       'MAIN_TOPIC_SCORE', 'SECONDARY_TOPIC', 'SECONDARY_TOPIC_SCR'],
      dtype='object')